Importar las librerias necesarias:


In [ ]:
# Herramientas de sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Modelos de sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

# Librerias complementarias
import warnings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar visualización y desactivar warnings
%matplotlib inline
warnings.filterwarnings("ignore")

## 1. Recolección de la data


Obtener el dataset.

In [ ]:
def get_dataset(file_path: str) -> pd.DataFrame:
    """
    Función para obtener un dataset en una ruta en especifica.
    
    Parametros:
        - file_path: La ruta al dataset.

    Devuelve:
    El dataset como un objeto DataFrame.
    """

    return pd.read_csv(file_path)

In [ ]:
df = get_dataset("raw_dataset.csv")

Estructura del dataset.

In [ ]:
df.head()

## 2. Preparación / preprocesamiento de la data


Herramientas para normalizar un dataset.

In [ ]:
# Escalador
SCALER = StandardScaler()

# Codificador
LABEL_ENCODER = LabelEncoder()

Normalizar el dataset.

In [ ]:
def get_normalized_dataset(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Normaliza un dataset proporcionado.

    Parametros:
        - df: Dataset a normalizar.

    Devuelve:
    El dataset normalizado.
    """

    # a. Eliminación de características redundantes o innecesarias
    dataset.drop_duplicates(inplace=True)

    # b. Limpieza de filas nulas, vacías o con error
    dataset.replace(["", " ", "?", "None", "N/A", "na"], pd.NA, inplace=True)
    dataset = dataset.dropna()
    dataset.reset_index(drop=True, inplace=True)

    # c. Encoder o codificador a las características no numéricas
    dataset["smoker"].replace({"yes": 1, "no": 0}, inplace=True)
    dataset["sex"].replace({"male": 1, "female": 0}, inplace=True)
    dataset["region"] = LABEL_ENCODER.fit_transform(dataset["region"])

    # d. Normalizar y estandarizar la data con un escalador de datos
    num_data = dataset.select_dtypes(include="number")
    scaled_data = SCALER.fit_transform(num_data)
    dataset = pd.DataFrame(scaled_data, columns=num_data.columns)

    return dataset

In [ ]:
df = get_normalized_dataset(df)

## 3. Análisis descriptivo de la data (EDA)


#### a. Analisis de la data con gráficas


1. Histograma de "charges"

* Esta gráfica permite visualizar la distribución de los gastos médicos y detectar posibles sesgos o outliers.

In [ ]:
sns.histplot(df['charges'], kde=True, color='blue', bins=10) # type: ignore
plt.title('Distribución de Charges')
plt.xlabel('Gastos Médicos')
plt.ylabel('Frecuencia')
plt.show()

2. Boxplot de "charges" segmentado por "smoker"

* Esta gráfica nos ayuda a comparar la distribución de los gastos médicos entre fumadores y no fumadores.

In [ ]:
sns.boxplot(x='smoker', y='charges', data=df, palette='Set2')
plt.title('Gastos Médicos por Estado de Fumador')
plt.xlabel('Fumador')
plt.ylabel('Gastos Médicos')
plt.show()

3. Scatter Plot: Relación entre "bmi" y "charges"

* En este diagrama se visualiza la relación entre el índice de masa corporal y los gastos médicos. 
* Se utiliza el color para diferenciar entre fumadores y no fumadores.

In [ ]:
sns.scatterplot(x='bmi', y='charges', hue='smoker', data=df, palette='Set1', s=100)
plt.title('Relación entre BMI y Gastos Médicos')
plt.xlabel('BMI')
plt.ylabel('Gastos Médicos')
plt.legend(title='Fumador')
plt.show()

4. Scatter Plot: Relación entre "age" y "charges"
* Permite explorar la posible relación entre la edad del paciente y el monto de los gastos médicos.

In [ ]:
sns.scatterplot(x='age', y='charges', hue='smoker', data=df, palette='coolwarm', s=100)
plt.title('Relación entre Edad y Gastos Médicos')
plt.xlabel('Edad')
plt.ylabel('Gastos Médicos')
plt.legend(title='Fumador')
plt.show()

5. Conteo de Frecuencia para Variables Categóricas
* Usamos countplots para ver la distribución de frecuencias de las variables categóricas, como "sex", "smoker" y "region".

In [ ]:
# Variable: Sex
plt.figure(figsize=(16, 4))
plt.subplot(1, 3, 1)
sns.countplot(x='sex', data=df, palette='pastel')
plt.title('Frecuencia por Sexo')
plt.show()

In [ ]:
# Variable: Smoker
plt.figure(figsize=(16, 4))
plt.subplot(1, 3, 2)
sns.countplot(x='smoker', data=df, palette='pastel')
plt.title('Frecuencia por Estado de Fumador')
plt.show()

In [ ]:
# Variable: Region
plt.figure(figsize=(32, 4))
plt.subplot(1, 3, 3)
sns.countplot(x='region', data=df, palette='pastel')
plt.title('Frecuencia por Región')
plt.tight_layout()
plt.show()

6. Heatmap de Correlaciones

* Este mapa de calor muestra la correlación entre las variables numéricas y permite identificar relaciones potenciales.

In [ ]:
correlation = df[['age', 'bmi', 'children', 'charges']].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de Calor de Correlaciones')
plt.show()

#### b. Interpretación las estadísticas de los datos


Estadísticas del dataset.

In [ ]:
df.describe()

##### Observaciones

1. Normalización evidente

Los datos de la media (mean) en cada variable son prácticamente cero, del mismo modo, los de la deviación estándar (std) están muy cerca a uno, lo cual muestra un claro proceso previo de normalización, que fue lo hicimos previamente en nuestro caso.
Esto nos será útil posteriormente para facilitar las comparaciones y el análisis exploratorio.

2. Registros consistentes

El número de registros (count) es consistente en cada variable, lo cual nos asegura que no se han perdido datos en el proceso.

3. Encoder presente

Los valores de las variables sex, smoker y region, se han codificado de modo que se ajusten a la misma escala que las demás variables.

Por ejemplo, en el caso de smoker, se puede ver que el percentil 25 y el valor mínimo (min) tienen valores fijos, lo que sugiere una codificación binaria con una asignación particular de valores que se ha transformado para cumplir con el estándar de media cero y varianza unitaria.

##### Conclusión

Estas estadísticas descriptivas permiten confirmar que el proceso de preprocesamiento y normalización se ha ejecutado correctamente.

## 4. Entrenamiento del modelo


Nombres de los modelos a entrenar.

In [ ]:
MODEL_NAMES = [
    "Ordinary Least Squares",
    "Ridge Regression",
    "Bayesian Regression",
    "Lasso Regression",
    "Nearest Neighbors Regression",
    "Random Forest Regression",
    "SVM Regression",
    "Neural Network MLP Regression",
]

##### Setup inicial

a. División del dataset en entradas y salidas/etiquetas (x, y).

In [ ]:
# Variables de entrada
X = df.drop("charges", axis=1)

# Variable de salida
y = df["charges"]

b. División del dataset en entrenamiento y testeo.

In [ ]:
# (80% entrenamiento, 20% testeo)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

##### Creación de los modelos

In [ ]:
# Ordinary Least Squares Regression
ols_model = LinearRegression()

# Ridge Regression
ridge_model = Ridge(alpha=1.0)

# Bayesian Regression
bayesian_model = BayesianRidge()

# Lasso Regression
lasso_model = Lasso(alpha=0.1)

# Nearest Neighbors Regression
knn_model = KNeighborsRegressor(n_neighbors=5)

# Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100, random_state=1, n_jobs=-1)

# SVM (Support Vector Machine) Regression
svm_model = SVR(kernel="rbf", C=1.0, epsilon=0.2)

# Neural Network MLP Regression
mlp_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=1)

In [ ]:
# Emparejar cada modelo con su nombre.
MODELS = {
    k: v
    for k, v in zip(
        MODEL_NAMES,
        [
            ols_model,
            ridge_model,
            bayesian_model,
            lasso_model,
            knn_model,
            rf_model,
            svm_model,
            mlp_model,
        ],
    )
}

Entrenamiento de cada modelo.

In [ ]:
for model in MODELS.values():
    model.fit(X_train, y_train)

## 5. Validación y testeo del modelo


#### a. Análisis de performance


Obtener las métricas de cada modelo.

In [ ]:
def get_all_metrics(_X_test=X_test, _y_test=y_test):
    """
    Calcula y devuelve las métricas de evaluación para
    todos los modelos entrenados.

    Parámetros:
    _X_test : pd.DataFrame, opcional
        Conjunto de datos de prueba con las variables
        predictoras. Por defecto usa "X_test".

    _y_test : pd.Series o np.array, opcional
        Conjunto de valores reales (etiquetas) correspondientes a
        las muestras de prueba. Por defecto usa "y_test".
    """

    return {
        model_name: {
            "R2": model.score(_X_test, _y_test),
            "MSE": mean_squared_error(_y_test, model.predict(_X_test)), # type: ignore
        }
        for model_name, model in MODELS.items()
    }

METRICS = get_all_metrics()

Configuración del dataframe.

In [ ]:
def get_metrics_df(metrics):
    """
    Convierte el diccionario de metricas en un DataFrame
    y renombra las columnas para una mayor claridad.
    """

    return (pd.DataFrame(metrics).T).rename(columns={"R2": "R² Score", "MSE": "MSE"})

metrics_df = get_metrics_df(METRICS)

##### - R² Score

In [ ]:
def show_R2(df: pd.DataFrame) -> None:
    """
    Muestra un gráfico de barras que compara el R² Score
    de cada modelo.

    Parámetros:
    df : pandas.DataFrame
        DataFrame que contiene, al menos, la columna "R² Score"
        y cuyos índices representan los nombres de los modelos.

    Retorna: None
        La función únicamente muestra el gráfico, sin retornar ningún valor.
    """

    plt.subplot(1, 2, 1)
    sns.barplot(x=df.index, y=df["R² Score"], palette="viridis")
    plt.title("Comparativa R² Score por Modelo")
    plt.xlabel("Modelo")
    plt.ylabel("R² Score")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

show_R2(metrics_df)

##### - MSE

In [ ]:
def show_MSE(df: pd.DataFrame) -> None:
    """
    Muestra un gráfico de barras que compara el MSE
    de cada modelo.

    Parámetros:
    df : pandas.DataFrame
        DataFrame que contiene al menos la columna "MSE" y
        cuyos índices representan los nombres de los modelos.

    Retorna: None
        La función únicamente muestra el gráfico, sin retornar
        ningún valor.
    """

    plt.subplot(1, 2, 2)
    sns.barplot(x=df.index, y=df["MSE"], palette="rocket_r")
    plt.title("Comparativa MSE por Modelo")
    plt.xlabel("Modelo")
    plt.ylabel("MSE")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

show_MSE(metrics_df)

#### b. Selección del algoritmo óptimo


##### Modelo seleccionado

El análisis de las métricas sugiere que el modelo de Neural Network MLP es el más adecuado para este problema.

In [ ]:
ALGORITMO_OPTIMO = mlp_model

##### Justificación

1. El R² Score es de 0.847072, el cual es el más alto de todos los modelos evaluados, es decir, este modelo explica aproximadamente el 84.7% de la variabilidad de los datos.

2. El MSE (Error Cuadrático Medio) es de 0.175550, el cual es el más bajo de todos los modelos evaluados.

3. Una red neuronal (MLP) es capaz de aprender y entender relaciones complejas y no lineales con los predictores, como es el caso de los efectos combinados de edad, bmi y hábito de fumar.

## 6. Despliegue del modelo y comprobación con data recién creada


##### a. Conversión de data nueva cruda a formato de entrada del algoritmo

1. Ruta al dataset de prueba.

In [ ]:
dataset_prueba = "dataset_prueba.csv"

2. Cargar y normalizar el dataset de prueba.

In [ ]:
df_test = get_dataset(dataset_prueba)
df_test = get_normalized_dataset(df_test)

# Conjuntos de datos
new_X_test = df_test.drop("charges", axis=1)
new_y_test = df_test["charges"]

##### b. Predicción de categoría del dato

Ejecutar la predicción.

In [ ]:
def predict() -> pd.DataFrame:
    """
    Genera predicciones utilizando los modelos entrenados
    para un conjunto de datos de prueba.
    
    Retorna: pandas.DataFrame
        Un DataFrame que contiene las predicciones de cada modelo (limitado a las primeras filas).
    """

    predictions = {}
    for model_name, model in MODELS.items():
        predictions[model_name] = model.predict(new_X_test)

    return pd.DataFrame(predictions).head()

In [ ]:
predict()

Mostrar el rendimiento de cada modelo.

In [ ]:
def show_performance() -> None:
    """
    Calcula y muestra las métricas de rendimiento de todos los
    modelos para un conjunto de datos de prueba.

    Retorna: None
    """

    metrics = get_all_metrics(new_X_test, new_y_test)
    metrics_df = get_metrics_df(metrics)
    show_R2(metrics_df)
    show_MSE(metrics_df)

In [ ]:
show_performance()